In [26]:
import pandas as pd
import numpy as np
import csv
import scipy
import sys
import os
import subprocess
from tqdm.notebook import tqdm

## Характеристики задачи

In [3]:
def find_features_tasks(df):
    '''
    Находит следующие характеристики для задачи:
    
    Число периодических программ
    Общая сложность периодических программ
    Общий период
    Число разделов
    Загрузка(отношение общего времени выполнения к общему периоду)
    Число интервалов
    
    Args:
        df(pd.DataFrame): Данные о постановке задачи построения статико-динамического
                          расписания
    Returns:
        int: число программ,
        int: общая сложность программ, 
        int: размер цикла планирования, 
        int: число разделов, 
        float: загрузка процессора этой задачей, 
        int: число интервалов
    '''
    Time = 1
    periods = df['period'].as_matrix()
    runtimes = df['complexity'].as_matrix()
    partitions = df['partition'].as_matrix()
    
    num_tasks = df.shape[0]
    num_part = len(np.unique(np.sort(partitions)))
    print("Num of partitions: ", num_part)
    for x in periods:
        Time = NOK(Time, x)
        
    counts = np.array([Time/x for x in periods])
    
    all_runtime = sum(runtimes[i]*counts[i] for i in range(len(df)))
    print("Complexity: ",all_runtime)
    
    intervals = set()
    intervals.add(0)
    for x in periods:
        y = x
        while(y <= Time):
            intervals.add(y);
            y += x;
            
    num_of_intervals = len(intervals) - 1;
    print("Runtime: ",Time)
    load = all_runtime / Time
    print('Load: ', load)
    print("Intervals: ", num_of_intervals)
    return tuple((num_tasks,all_runtime, Time, num_part, load, num_of_intervals))

def find_features_works(df):
    '''
    Находит следующие характеристики для задачи
    Число задач
    Общее время выполнения
    Общий период
    Число разделов
    Загрузка(отношение общего времени выполнения к общему периоду)
    Число интервалов
    '''
    
    starts = df['start'].as_matrix()
    finishes = df['finish'].as_matrix()
    runtimes = df['runtime'].as_matrix()
    partitions = df['partition'].as_matrix()
    
    num_tasks = df.shape[0]
    num_part = len(np.unique(np.sort(partitions)))
    #print("Num of partitions: ", num_part)

    Time = np.max(finishes)   
    #counts = np.array([Time/x for x in periods])
    
    all_runtime = np.sum(runtimes)
    
    #print("All_runtime: ",all_runtime)
    
    intervals = set()
    intervals.add(0)
    for x in starts:
        intervals.add(x)
    for x in finishes:
        intervals.add(x)          
    num_of_intervals = len(intervals) - 1;
    
    #print("All time: ",Time)
    
    load = all_runtime / Time
    
    average_runtime = all_runtime/ num_tasks
    #print('Load: ', load)
    #print("Число интервалов: ", num_of_intervals)
    return tuple((num_tasks, average_runtime, num_part, load, num_of_intervals))
      
    

In [4]:
def NOD(a, b):
    while a != 0 and b != 0:
        if a > b:
            a %= b
        else:
            b %= a
    return a + b


def NOK(a, b):
    return a*b / NOD(a, b)

def NOKA(a):
    first = a[0]
    for i in range(1, len(a)):
        second = a[i]
        first = first*second / NOD(first, second)
    
    return first


---

# Обработка решения

In [5]:
def get_time_perfomance(file):
    time = 0.
    perfomance = 0.
    with open(file, 'r') as f:
        str_ = f.readline(); # Пропускаем заголовок
        str_ = f.readline();
        time = float(str_.split(',')[0])
        perfomance = float(str_.split(',')[2])
    return time, perfomance

In [110]:
def compute_cost(solution, system):
    """
    Возвращает стоимость полученной системы
    Args:
        solution (`class`pd.DataFrame): Построенное решение
        system (`class`pd.DataFrame): На основе какой системы строилось
    Returns:
        int: Стоимость системы
    """
    procs = solution.groupby(['Processor', ' Processor Type']).count().index
    cost = 0
    for proc in procs:
        cost += system.iloc[proc[1]]['Cost']
    return cost

---

# Периодические работы

## Равномерное распределение параметров задач

In [7]:
def create_tasks_random(tasks_bound, n_part, periods, load, system):
    '''
    Create one task for static-dynamic shedule с периодическими интервалами 
    по равномерному распределению
    Args:
        n_tasks(int): Число периодических программ в задаче
        n_part(int): число разделов в задаче
        periods(list(int)): возможные значения для периодов программ
        load(float): приблизительная нагрузка полученной задачи
        system(pd.DataFrame): Система в виде датафрейма
    
    Returns:
        pd.DataFrame: Постановка задачи в табличном формате с колонками
                      'partition', 'complexity', 'period', 'functionalities','cost'
    '''
    # Интервал планирования
    time_circle = NOKA(periods)
    
    # процессоры
    procs = list(system.index)
    
    # Разделы
    all_parts = np.array(range(1, n_part + 1))
    
    # готовим знание о системе
    options = set()
    for _, row in system.iterrows():
        options = options.union(set(row.Options.split(';')))
    # каждому процессору его перфоманс приписываем
    
    perf = {}
    for proc in procs:
        perf[proc] = system.iloc[proc].Performance * time_circle
        
    # Привяжем рандомно разделы к процессорам и свяжем опции
    bound = {}
    tasks = {}
    d_part_options = {}
    for part in all_parts:
        r = np.random.randint(len(procs))
        bound[part] = procs[r]
        # Определяем число задач в разделе
        tasks[part] = np.random.randint(tasks_bound[0], tasks_bound[1])
        d_part_options[part] = set(system.iloc[procs[r]].Options.split(';'))
        d_part_options[part] = ';'.join(d_part_options[part])
    #print( 'Распределение разделов:', bound)
    
    # Считаем число разделов по процессорам
    tasks_proc = {}
    for i,task in tasks.items():
        key = bound[i]
        if key in tasks_proc:
            tasks_proc[key].add(i)
        else:
            tasks_proc[key] = set({i})
        
    # Считаем процент загрузки для каждого раздела 
    part_load = {}
    for proc, parts in tasks_proc.items():
        a = np.random.sample(len(parts))
        a = a/a.sum()
        for i, part in enumerate(parts):
            part_load[part] = a[i]
    #print('Загрузка раздела:', part_load)
    
    df_all = pd.DataFrame()
    
    np.random.shuffle(all_parts)
    for part in all_parts:
        proc_cur = bound[part]
        performance = perf[proc_cur]
        rows = []
        cur_part_load = part_load[part]
        tasks_cur = tasks[part]
        a = np.random.sample(tasks_cur)
        a = a/a.sum()
        for i in range(tasks_cur):
            row = []
            np.random.shuffle(periods)      
            row.append(performance*load*a[i]*cur_part_load // (time_circle // periods[0]) )
            row.append(periods[0])
            rows.append(row)
        df = pd.DataFrame(rows, columns=['complexity', 'period'])
        df['partition'] = part
        df_all = df_all.append(df)
        
        
    df_all['left'] = 0
    df_all['right'] = df_all['period']
    df_all['options'] = df_all['partition'].map(d_part_options)
    df_all['complexity'] = df_all['complexity'].astype(int)
    return df_all[['partition','complexity','period','left','right','options']]
    

 -----------------
# Обычные работы


---

# Расширение данных для задачи синтеза

In [33]:
system2

,Performance,Options,Cost
0,4,o1,10
1,1,o2;o1,3


In [43]:
system2.append(pd.DataFrame([[2,4,5]], columns=system2.columns)).reset_index(drop=True)

,Performance,Options,Cost
0,4,o1,10
1,1,o2;o1,3
2,2,4,5


In [61]:
np.random.choice(range(system2.shape[0]))

0

In [62]:
system2.columns

Index(['Performance', 'Options', 'Cost'], dtype='object')

In [131]:
def extend(system, num_proc):
    """
    Расширяет переданную ей системы до нужного число процессоров так,
    чтобы на переданной системе достигался минимум по стоимости для выполнения расписания на нем
    Возвращает нужные набор программ для построения расписания
    
    Args:
        system ('class'pd.DataFrame): Описание минимальной системы
        num_proc (int): Требуемое число процессоров
    
    Return:
        'class'pd.DataFrame: Процессорная система
        
        'class'pd.DataFrame: Программы для составления расписаний
    
    
    """
    num_proc_add = num_proc - system.shape[0] 
    if num_proc_add < 0:
        print('Number of processors are too low')
        return
    
    rows = []
    for i in range(num_proc_add):
        base_row = np.random.choice(range(system.shape[0]))
        base_perf = system.iloc[base_row]['Performance']
        base_opt = system.iloc[base_row]['Options']
        base_cost = system.loc[base_row]['Cost']
        r = 0.1 +2*np.random.random()
        new_perf = round(base_perf*r);
        if new_perf == 0:
            new_perf = 1;
        new_opt = base_opt;
        new_cost = round(base_cost*r);
        rows.append([new_perf, new_opt, new_cost])
        
    system_ext =system.append(pd.DataFrame(rows, columns=system.columns)).reset_index(drop=True)
    
    tasks = create_tasks_random(tasks_bound=[2,5], n_part=system.shape[0]+2, 
                                periods=[1000, 200, 500], load=0.8, 
                                system=system)
    return system_ext, tasks
        
     

In [134]:
df1, df2 = extend(system2, 10)

In [135]:
df1

,Performance,Options,Cost
0,4.0,o1,10.0
1,1.0,o2;o1,3.0
2,1.0,o1,2.0
3,4.0,o1,9.0
4,2.0,o1,5.0
5,4.0,o1,9.0
6,7.0,o1,17.0
7,5.0,o1,13.0
8,1.0,o2;o1,1.0
9,6.0,o1,15.0


In [75]:
df2

,partition,complexity,period,left,right,options
0,1,104,500,0,500,o2;o1
1,1,41,200,0,200,o2;o1
0,2,53,500,0,500,o2;o1
1,2,68,500,0,500,o2;o1
2,2,5,500,0,500,o2;o1
3,2,63,500,0,500,o2;o1
0,3,170,200,0,200,o1
1,3,23,1000,0,1000,o1
0,4,963,1000,0,1000,o1
1,4,272,200,0,200,o1


---

# Считывание систем

Всего есть в папке systems

In [14]:
system_path = os.path.join('systems', 'system2')
system2 = pd.read_csv(system_path, sep='\t', index_col=[0])

system_path = os.path.join('systems', 'system4')
system4 = pd.read_csv(system_path, sep='\t', index_col=[0])

system_path = os.path.join('systems', 'system8')
system8 = pd.read_csv(system_path, sep='\t', index_col=[0])

# Проведение тестирования

## Расписания

In [27]:
seed = 17
np.random.seed(seed)

Генерация тестов на лету будет (без повторения тогда получится)

In [28]:
n_iter = 10
configs = []
configs.append((3, [500, 1000, 2000],(2,5), system2, 0.6))
configs.append((5, [500, 1000, 2000],(2,5), system2, 0.7))
configs.append((7, [500, 1000, 2000],(2,5), system2, 0.8))

configs.append((5, [500, 1000, 2000],(2,5), system4, 0.6))
configs.append((7, [500, 1000, 2000],(2,5), system4, 0.7))
configs.append((9, [500, 1000, 2000],(2,5), system4, 0.8))

configs.append((9, [500, 1000, 2000],(2,5), system8, 0.6))
configs.append((11, [500, 1000, 2000],(2,5), system8, 0.7))
configs.append((13, [500, 1000, 2000],(2,5), system8, 0.8))

In [29]:
%%time
rows = []
for config in tqdm(configs):
    n_part = config[0]
    periods = config[1]
    tasks_bound = config[2]
    system = config[3]
    load = config[4]
    for i in range(n_iter):
        df_test = create_tasks_random(tasks_bound=tasks_bound, n_part=n_part, periods=periods, load=load, system=system)
        df_test.to_csv(f'tasks/tmp.csv', sep='\t')
        result = subprocess.call([f'./scheduler', '1', f'tasks/tmp.csv', f'systems/system{system.shape[0]}' ,'schedule'])
        time, performance = get_time_perfomance(f'tasks/tmp.res')
        row = (n_part, periods, tasks_bound, system.shape[0], load, time, performance)
        rows.append(row)


Wall time: 58.2 s


In [30]:
df_res = pd.DataFrame(rows, columns=['Число разделов', 'Периоды', 'Количество задач', 'Система', 'Загрузка', 'Время', 'Эффективность'])

In [31]:
df_res

,Число разделов,Периоды,Количество задач,Система,Загрузка,Время,Эффективность
0,3,"[1000, 2000, 500]","(2, 5)",2,0.6,0.528,1.0
1,5,"[1000, 2000, 500]","(2, 5)",2,0.7,1.477,1.0
2,7,"[1000, 2000, 500]","(2, 5)",2,0.8,2.017,1.0
3,5,"[1000, 500, 2000]","(2, 5)",4,0.6,1.699,1.0
4,7,"[500, 1000, 2000]","(2, 5)",4,0.7,4.790,1.0
5,9,"[1000, 2000, 500]","(2, 5)",4,0.8,5.827,1.0
6,9,"[2000, 1000, 500]","(2, 5)",8,0.6,10.609,1.0
7,11,"[1000, 500, 2000]","(2, 5)",8,0.7,13.723,1.0
8,13,"[1000, 500, 2000]","(2, 5)",8,0.8,15.650,1.0


In [32]:
df_res.groupby(['Система', 'Загрузка', 'Число разделов'])[['Время','Эффективность']].mean()

Время  Эффективность
Система Загрузка Число разделов                       
2       0.6      3                0.528            1.0
        0.7      5                1.477            1.0
        0.8      7                2.017            1.0
4       0.6      5                1.699            1.0
        0.7      7                4.790            1.0
        0.8      9                5.827            1.0
8       0.6      9               10.609            1.0
        0.7      11              13.723            1.0
        0.8      13              15.650            1.0

## Синтез

In [140]:
n_iter = 10
configs = []
configs.append([system2, 10, 13])

In [144]:
%%time
rows = []
for config in tqdm(configs):
    system = config[0]
    num_proc = config[1]
    expected_cost = config[2]
    for i in range(n_iter):
        df_sys, df_task = extend(system, num_proc)
        df_sys.to_csv(f'systems/tmp.csv', sep='\t')
        df_task.to_csv(f'tasks/tmp.csv', sep='\t')
        result = subprocess.call([f'./scheduler', '1', f'tasks/tmp.csv', f'systems/tmp.csv' ,'synthesis'])
        time, performance = get_time_perfomance(f'tasks/tmp.res')
        solution = pd.read_csv('tasks/tmp.sol')
        cost = compute_cost(solution, df_sys)
        row = (time, performance, expected_cost, cost)
        rows.append(row)


Wall time: 5min 14s


In [145]:
df_res = pd.DataFrame(rows, columns=['Время', 'Эффективность', 'Ожидаемая стоимость', 'Стоимость'])

In [146]:
df_res

,Время,Эффективность,Ожидаемая стоимость,Стоимость
0,19.497000,1.0,13,10.0
1,18.101000,1.0,13,10.0
2,51.931999,1.0,13,0.0
3,41.554001,1.0,13,11.0
4,30.240999,1.0,13,11.0
5,16.215000,1.0,13,10.0
6,36.838001,1.0,13,11.0
7,20.063000,1.0,13,13.0
8,41.729000,1.0,13,11.0
9,36.562000,1.0,13,10.0
